In [1]:
import pandas as pd

In [3]:
!bzcat bookings.csv.bz2 | head -10000 > bookings_sample_10k.csv


bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = bookings.csv.bz2, output file = (stdout)


In [4]:
!bzip2 -f bookings_sample_10k.csv

In [5]:
!bzcat bookings_sample_10k.csv.bz2 | wc

   10000  279626 4232732


In [32]:
df1 = pd.read_csv('bookings_sample_10k.csv.bz2', sep='^')

In [47]:
df1.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,...,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


In [68]:
df1_filtered = df1[['arr_port', 'pax']]
df1_group = df1_filtered.groupby(by='arr_port').sum()
df1_top10 = df1_group.sort_values('pax', 
                                  ascending=False).reset_index().head(10)


In [69]:
df1_top10

,arr_port,pax
0,HKG,112
1,LGA,95
2,ORD,94
3,JFK,92
4,LAX,91
5,SFO,91
6,MCO,90
7,DCA,82
8,DEN,79
9,LHR,76


In [93]:
dataframe_bookings = pd.read_csv(
                                'bookings_sample_10k.csv.bz2',
                                chunksize = 1000,
                                engine = 'c',
                                iterator = True,
                                compression = 'bz2'
                                                            
                                )


In [94]:
rows_total = 0
for df in dataframe_bookings:
    bookings_rows = df.shape[0]
    rows_total += bookings_rows
print(rows_total)

9999


Top 10 arrival airports in the world in 2013 (using the bookings file)
• Arrival airport is the column arr_port. It is the IATA code for the airport
• To get the total number of passengers for an airport, you can sum the column pax, grouping by arr_port. Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the canceled bookings).
• Print the top 10 arrival airports in the standard output, including the number of passengers.
• Bonus point: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at GeoBases in Github)
• Bonus point: Solve this problem using pandas (instead of any other approach

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,...,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


 Exercise 3
Plot the monthly number of searches for flights arriving at Málaga, Madrid or Barcelona
• For the arriving airport, you can use the Destination column in the searches file.
• Plot a curve for Málaga, another one for Madrid, and another one for Barcelona, in the same figure.
• Bonus point: Solving this problem using pandas (instead of any other approach)

In [97]:
dataframe_bookings_total = pd.read_csv(
                                'bookings.csv.bz2',
                                chunksize = 10000,
                                engine = 'c',
                                iterator = True,
                                compression = 'bz2'
                                                            
                                )


In [98]:
rows_total = 0
for df in dataframe_bookings_total:
    bookings_rows = df.shape[0]
    rows_total += bookings_rows
print(rows_total)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 5000009, saw 2
